In [ ]:
%load_ext autoreload 
%autoreload 2

# 0. Loading dataset and libraries

**General note:** to make this notebook readable (and runnable), we present here the results, and not the whole process, of our endeavors. This notebook follows the logic of our [data story](https://epfl-ada.github.io/ada-2024-project-laambada/). For implementation details, please refer to scripts and notebooks found in `.\src`

## 0.1 Load libraries

In [82]:
import numpy as np
import pandas as pd
from src.scripts.load_and_save import load_data

## 0.2 Load data

This is a cleaned version of the BindingDB. It contains entries with the family of targets we chose.

In [83]:
df = load_data()
df.head()

c:\Users\maudd\Coding\MA3\Ada\new\ada-2024-project-laambada\src\scripts\load_and_save.py:17: DtypeWarning:

Columns (17,20,21,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.



,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,Charge,Aliphatic OH,Aromatic NH,Ester,Ether,Amide,Ketone,Benzene Ring,pIC50,pKi
0,2880,Cn1ncc2c(cccc12)-c1cccn2nc(Nc3ccc4CCN(CCc4c3)C...,"InChI=1S/C29H31N7O2/c1-29(2,3)38-28(37)35-15-1...",MAZKPYXDQCNDAZ-UHFFFAOYSA-N,2013,"US8501936, 298::US8501936, 300",Tyrosine-protein kinase JAK2,Homo sapiens,0.24,NaN,...,0,0,0,0,1,1,0,2,NaN,9.619789
1,2953,Cn1ncc2c(cccc12)-c1cccn2nc(Nc3ccc4CCN(CCc4c3)C...,"InChI=1S/C29H31N7O2/c1-29(2,3)38-28(37)35-15-1...",MAZKPYXDQCNDAZ-UHFFFAOYSA-N,2013,"US8501936, 298::US8501936, 300",Tyrosine-protein kinase JAK3,Homo sapiens,0.25,NaN,...,0,0,0,0,1,1,0,2,NaN,9.602060
2,3039,Cn1ncc2c(cccc12)-c1cccn2nc(Nc3ccc4CCN(CCc4c3)C...,"InChI=1S/C29H31N7O2/c1-29(2,3)38-28(37)35-15-1...",MAZKPYXDQCNDAZ-UHFFFAOYSA-N,2013,"US8501936, 298::US8501936, 300",Tyrosine-protein kinase JAK2,Homo sapiens,0.41,NaN,...,0,0,0,0,1,1,0,2,NaN,9.387216
3,3041,CS(=O)(=O)c1ccc(cc1)C(=C)n1nc(NC(=O)Nc2ccc(cc2...,InChI=1S/C24H27N7O3S/c1-4-22-27-23(29-31(22)17...,IMNDRGLFDWUUKR-UHFFFAOYSA-N,2061,"US8501936, 90",Tyrosine-protein kinase JAK2,Homo sapiens,0.80,NaN,...,0,0,0,0,0,2,0,2,NaN,9.096910
4,3072,CS(=O)(=O)c1ccc(cc1)C(=C)n1nc(NC(=O)Nc2ccc(cc2...,InChI=1S/C24H27N7O3S/c1-4-22-27-23(29-31(22)17...,IMNDRGLFDWUUKR-UHFFFAOYSA-N,2061,"US8501936, 90",Tyrosine-protein kinase JAK3,Homo sapiens,0.99,NaN,...,0,0,0,0,0,2,0,2,NaN,9.004365


# 1. Data story contents

## 1.1 Kinases looking for love
This part serves to introduce the targets belonging to the TYROSINE-PROTEIN KINASE HOPSCOTCH family, further refered to as tyrosine kinase family.

Previously, we selected the targets belonging to the tyrosine kinase family to conduct our analyses. This choice was primarily motivated by a manageable amount of targets, accompanied by a decent amount of ligands per target, a requirment for ML-based processing.

In [84]:
from src.utils.embeddings_plots import  reduce_family

# Let's reduce the number of targets
df['Target Name Detailed'] = df['Target Name']
df['Target Name'] =  df['Target Name'].apply( reduce_family)

## 1.2 What is a good match?
This part serves to motivate our choice of metrics that can be used to characterize a binding.

### 1.2.1 Metrics
We start by exploring some metrics provided by the dataset.

In [85]:
# Usefull metrics 
metric_percent = {}
for col in ['Ki (nM)', 'IC50 (nM)', 'Kd (nM)','kon (M-1-s-1)', 'koff (s-1)', 'EC50 (nM)', 'pH', 'Temp (C)']:
    metric_percent[col] = len(df[col].dropna())/len(df)


import plotly.express as px

x =list(metric_percent.keys())
y=list(metric_percent.values())

fig = px.bar(x=x, y=y)

fig.update_layout(xaxis_title='', yaxis_title='Fraction of Nan')

fig.show()

fig.write_html('src/data/figures/nan_fraction.html')

At this point, we drop `kon (M-1-s-1)`, `koff (s-1)`, `EC50 (nM)` columns, as there is a lot of missing data and these metrics by themselves do not bring enough information about the binding.

Both `IC50 (nM)` and `Ki (nM)` provide meaningful information. `Ki (nM)` is more interpretable for our project, as it doesn't depend on specific reaction conditions like `Ki (nM)` does. However, there is much more data for `IC50 (nM)`, so we'll see how we'll handle that. We will actually use the -log10 of these metrics, as they are more interpretable: pIC50 and pKi. The scale is such that the higher the value, the better the binding, is more adapted to the concentration range of this data, and is commonly used in the literature.

_Side note: BindingDB is drug discovery-oriented, so we expected to see more IC50 metrics_ 

As for `pH` and `Temp (C)`, we keep them both for later analyses. 

`Kd (nM)` column doesn't have enough data (even if we calculate it ourselves using `kon (M-1-s-1)` and `koff (s-1)`, so we remove it.

In [62]:
metrics = ['pKi', 'pIC50'] 
features = ['pH', 'Temp (C)']

In [63]:
# Generating the pKi and pIC50 columns
for metric in metrics : 
    df['p' + metric] = np.where(
        df[metric] > 0,  # Only apply log10 to positive values
        -np.log10(df[metric] * 1e-9),  # Transform to molar and take -log10
        np.nan  # Assign NaN for zero or negative values
    )

Even if `IC50 (nM)` can be less meaningful due to its dependency on essay conditions, it is highly correlated to `Ki (nM)`. One potential explanation is a certain uniformity in experiment conducted on the same target. Therefore, it looks like we can extract meaningfull information about the inhibitory potential from `IC50 (nM)`. 

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

df_Ki_IC50 = df.dropna(subset=['pKi', 'pIC50'])

# Pearson Correlation
pearson_corr = df_Ki_IC50[['pKi', 'pIC50']].corr(method='pearson').iloc[0, 1]

# Linear Regression
reg = LinearRegression()
reg.fit(df_Ki_IC50['pIC50'].values.reshape(-1, 1), df_Ki_IC50['pKi'].values.reshape(-1, 1))
df_Ki_IC50['pred_pki'] = reg.predict(df_Ki_IC50['pIC50'].values.reshape(-1, 1))
linear_coef = reg.coef_[0][0].round(4)

# Line and Scatter Plot
fig1 = px.line(df_Ki_IC50, x='pIC50', y='pred_pki')
fig1.update_traces(line_color='red')
fig2 = px.scatter(df_Ki_IC50, x='pIC50', y='pKi', color='Target Name')
fig3 = go.Figure(data=fig1.data + fig2.data)

# Adding Annotations
fig3.add_annotation(
    x=df_Ki_IC50['pIC50'].max()-0.5,
    y=df_Ki_IC50['pKi'].min()+0.5,
    text=f"Pearson Corr: {pearson_corr:.2f}<br>Linear Coef: {linear_coef:.2f}",
    showarrow=False,
    align="right",
    font=dict(size=14, color="black")
)

# Layout updates
fig3.update_layout(
    xaxis_title="pIC50",
    yaxis_title="pKi"
)

# Show plot
fig3.show()
fig3.write_html('src/data/figures/pKi_pIC50.html')

### 1.2.1 Chemical characterization
Now we move to the chemical characterization of ligands obtained through RDKit Descriptors. The two main descriptors we are interested in are Molecular Weight, LogP and the functionnal group count, important drug properties. We will do some visual exploration of these descriptors. 

In [65]:
properties = ['Ligand MW', 'logP']	

n_metrics = len(metrics)
n_properties = len(properties)

In [ ]:
df_ic50 = df.dropna(subset='IC50 (nM)')
df_ki = df.dropna(subset='Ki (nM)')

def get_metrics_df(df, metric):
    return df.dropna(subset=metric)

len(df_ic50)

In [ ]:
from src.utils.exploration_and_clean import plot_chemical_property_distributions

properties_colors = ['#EB89B5', '#330C73']
plot_chemical_property_distributions(df, metrics, chemical_properties=properties, properties_colors=properties_colors, filepath='src/data/figures/chemchar1.html', plot_metrics=True) 


In [ ]:
df.head()
functionnal_groups = ["Aliphatic OH", "Aromatic NH", "Ester", "Ether", "Amide",	"Ketone", "Benzene Ring"]
df_functionnal_groups = df[functionnal_groups]
df_functionnal_groups.head()

In [69]:
counts = df_functionnal_groups.sum()

In [70]:
fig = px.histogram(x=functionnal_groups, y=counts)
fig.update_layout(xaxis_title= '', yaxis_title='Count')
fig.write_html('src/data/figures/functional_groups.html')

### 1.2.2 Relation betweeen chemical properties and metrics
Here comes our first question: is there a relation between the chemical properties of the ligands and the binding metrics? We will explore this question by looking at the correlation between the metrics and the chemical properties.

In [ ]:
metric = metrics[0]
property = properties[0]
fig = px.scatter(get_metrics_df(df, metric), x=property, y=metric, marginal_x="histogram", marginal_y="histogram", color='Target Name')
fig.update_layout(height=600, width=1200)
fig.show()

In [ ]:
metric = metrics[1]
property = properties[0]
fig = px.scatter(get_metrics_df(df, metric), x=property, y=metric, marginal_x="histogram", marginal_y="histogram", color='Target Name')
fig.update_layout(height=600, width=1700)
#fig.add_hline(y=1e3, line_dash="dash",row=1, col=1)
fig.show()

In [ ]:
metric = metrics[0]
property = properties[1]
fig = px.scatter(get_metrics_df(df, metric), x=property, y=metric, marginal_x="histogram", marginal_y="histogram", color='Target Name')
fig.update_layout(height=600, width=1200)
fig.show()

In [ ]:
metric = metrics[1]
property = properties[1]
fig = px.scatter(get_metrics_df(df, metric), x=property, y=metric, marginal_x="histogram", marginal_y="histogram", color='Target Name')
fig.update_layout(height=600, width=1700)
fig.show()

These metrics do not seem to have a simple linear relation with the chemical properties. We will look at the mutual information between the metrics and the chemical properties to see if there is a non-linear relation. We will slightly change the properties of interest. These are decided based on the properties contributing the most to the first component of the PCA of the RDKit descriptors, that is done later in the notebook.

In [99]:
# Now verify the Mutual information between the chemical properties and the metrics
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler


metric = metrics[0]
df_embeddings = load_data('src/data/embeddings_RDKIT_descriptors.csv.zip')
df_predict = get_metrics_df(df.merge(df_embeddings, on='Ligand SMILES', how='left'), metric)

features_of_interest = ["NumValenceElectrons", "ExactMolWt", "Chi0", "HeavyAtomCount"]

X = df_predict[features_of_interest]
y = df_predict[metric]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)



df_MI = pd.DataFrame(mutual_info_regression(X_scaled, y), index=features_of_interest, columns=['Mutual Information'])
df_MI

,Mutual Information
NumValenceElectrons,0.047122
ExactMolWt,0.053231
Chi0,0.066305
HeavyAtomCount,0.025693


The mutual information between the metrics and the chemical properties is close to zero. This suggests that there is no simple relation between the chemical properties and the binding metrics. This motivates the use of embeddings to try to have a more complex representation of the ligands.

### 1.2.3 Exploring the ligands a bit more


In [106]:
# If the ligands are present several times in the dataset, are their metrics consistent?

# Step 1: Group by 'Ligand SMILES' and filter for groups with at least 2 entries
filtered_ligands = (
    df
    .groupby('Ligand SMILES')
    .filter(lambda x: len(x) > 25)  # Keep groups with at least 25 entries
)

# Step 2: Group again and calculate mean and standard deviation
ligand_stats = (
    filtered_ligands
    .groupby('Ligand SMILES')['pIC50']
    .agg(['mean', 'std'])  # Calculate both mean and standard deviation
)

print(len(ligand_stats))
# Step 3: Add the number of target they have been tested on
ligand_stats['n_targets'] = [len(group_df['Target Name'].unique()) for _, group_df in filtered_ligands.groupby('Ligand SMILES')]

# Display or use the result
ligand_stats.head()

45


,mean,std,n_targets
Ligand SMILES,,,
CC(C)NC(=O)c1cnc(cn1)N1CC(CC#N)(C1)n1cc(cn1)-c1ccnc2[nH]ccc12,7.115528,1.565333,3
CC(C)NC(=O)c1cnc(cn1)N1CC(CC#N)(C1)n1cc(cn1)-c1ncnc2[nH]ccc12,6.577917,1.326089,3
CC(CC#N)N1[C@H]2CC[C@@H]1CN(C2)c1ccnc(Nc2cnn(C)c2)n1,6.105553,0.844857,4
CCC(C)N1CC(C1)N1CCc2[nH]c(nc2C1)-c1n[nH]c2cc(ccc12)-c1cc(F)c(O)cc1CC,4.976735,0.604735,4
CCCN1Cc2[nH]c(nc2C[C@H]1C(=O)N1CC(C)(C1)N(C)C)-c1n[nH]c2cc(ccc12)-c1cc(F)c(O)cc1CC,7.697688,1.191497,4


In [107]:
import plotly.graph_objects as go

# Sort the ligands by mean pIC50 values
top_ligands_sorted = top_ligands.sort_values(by='mean', ascending=False)

# Replace long Ligand SMILES with simple ligand names or indices
ligand_names = [f'Ligand {i+1}' for i in range(len(top_ligands_sorted))]

# Create the bar plot with error bars
fig = go.Figure(data=[
    go.Bar(
        #x=ligand_names,
        y=top_ligands_sorted['mean'],
        error_y=dict(type='data', array=top_ligands_sorted['std'], visible=True),
        marker=dict(color='skyblue')
    )
])

# Update layout for better readability
fig.update_layout(
    xaxis_title='Ligands with more than 25 matches',
    yaxis_title='Mean pIC50',
    xaxis=dict(tickangle=45),
    showlegend=False,
    template='plotly_white'
)

# Show the plot
fig.show()

## 1.3 Match made in heaven
This section explores various ways of embedding ligands: 
* RDKit descriptors, containing information about the chemical structure of the ligands
* Morgan fingerprints
* Mol2Vec embeddings, based on a pre-trained model
* The combination of the above

These embeddings are generated with the scripts found in `.\src\embeddings.py`, as it is computationally and time expensive, we save them and load them here.

In [75]:
from src.run_reduction import run_analysis

path_RD_df  = 'src/data/embeddings_RDKIT_descriptors.csv.zip'
path_Mol2vec_df = 'src/data/embeddings_Mol2Vec.csv.zip'
path_Morgan_df = 'src/data/embeddings_Morgan_Fingerprint.csv.zip'
path_full_df = 'src/data/embeddings_full.csv.zip'


### 1.3.1 RDKit Descriptors space
In an intuitive approach, we constructed our first "embedding space" by putting all RDKit Descriptors together. After some processing and dimensionality reduction, this is what we obtained.

In [ ]:
run_analysis(df, path_RD_df, do_umap=True)

In [ ]:
# Let's do more chemical characterization!
import random

features_of_interest = ["NumValenceElectrons", "ExactMolWt", "MolWt", "Chi0", "LabuteASA", "Kappa1", "HeavyAtomMolWt", "HeavyAtomCount", "Chi0v", "Chi1", "SPS", "FractionCSP3", "NumSaturatedRings", "NumAliphaticRings", "BCUT2D_CHGLO", "BCUT2D_CHGHI", "BCUT2D_LOGPLOW", "SlogP_VSA6", "SMR_VSA7", "NumAromaticCarbocycles"]
colors = ["#{:06x}".format(random.randint(0, 0xFFFFFF)) for _ in features_of_interest]

df_embeddings = load_data('src/data/embeddings_RDKIT_descriptors.csv.zip')
plot_chemical_property_distributions(df, metrics, chemical_properties=features_of_interest[:5], properties_colors=colors[:5], filepath='src/data/chemchar2.html', plot_metrics=False, df_embeddings=df_embeddings) 

### 1.3.2. [Mol2vec](https://pubs.acs.org/doi/10.1021/acs.jcim.7b00616) embedding space
In a quest for a more homogeneous embedding, we tried out Mol2vec. Mol2vec is an unsupervised machine learning approach to learn vector representations of molecular substructures. Compounds are encoded as vectors by summing the vectors of the individual substructures. The direction of substructure vectors created by Mol2vec provides information about chemically related substructures. 

In [ ]:
run_analysis(df, path_Mol2vec_df, do_umap=True)

### 1.3.3. Morgan fingerprints space
We next interrogated Morgan fingerprints, a type of molecular fingerprints. They are based on the idea of transforming local structural information of a molecule into a fixed-length bit string. This representation captures information about the molecular structure in a way that is useful for comparing molecules or identifying structural similarity.

In [ ]:
run_analysis(df, path_Morgan_df, do_umap=True)


### 1.3.4. Combined embeddings
Finally, we combine all three embeddings tested until that.

In [ ]:
run_analysis(df, path_full_df, do_umap=True)